# 7월 14일

- 오늘까지 이론 마무리
- 실습 환경 구축
- 네트워크 관제사 
    - 보안관제팀: NIDS
    - 침해 대응팀: CERT

## 와이어샤크
- 와이어샤크는 일반적으로 **패킷 분석용**으로 사용.
    - 패킷 수집도 가능하지만, 수집은 주로 다른 앱을 사용한다.(TCPdump, TShark)
- 분석
    - 번역 방식: 수집을 끝내고 한 번에 분석. 컴파일러 방식
    - 통역 방식: 바로바로 분석하는 방식. 인터프린터 방식
        - 와이어 샤크는 인터프린터 방식 사용
        - GUI 기반, 패킷 수집과 동시에 분석 --> 수집률 저하
        - **모든 패킷을 수집하지 못한다!**
        - 수집은 CUI 기반으로 수집만 할 수 있는 프로그램을 사용.


### 주소 기반의 트래픽 수집
- 캡처 필터
- 캡처 필터는 처음부터 세세한 조건을 거는 것을 권장하지 않음.
    - HTTPS 패킷을 분석하려고 해도 TCP로 필터링하는 것이 좋다.

1. 특정 주소에서/로 오는 트래픽 수집 가능(wireshark PDF, page21~26)
    - host (IP): 송수신지 상관 없이 IP주소에 대한 트래픽
    - host 2406:da00:ff00::6b16:f02d: IPv6 타입
    - not host 10.3.1.1
    - src host 10.3.1.1
    - dst host 10.3.1.1
    - host 10.3.1.1 or host 10.3.1.2
    - host www.espn.com

2. IP 주소 범위에서/로 오는 트래픽 수집
    - net 10.3.0.0/16   (이에 해당하는 네트워크망으로 오고 나가는 패킷 수집)
    - net 10.3.0.0 mask 255.255.0.0
    - ipv6 net 2406:da00:ff00::/64
    - not dst net 10.3.0.0/16
    - dst net 10.3.0.0/16
    - src net 10.3.0.0/16

3. 브로드캐스트 또는 멀티캐스트 트래픽 수집
    - ip broadcast
    - ip multicast
    - dst host ff02::1
    - dst host ff02::2

4. MAC 주소 기반의 트래픽 수집
    - ether host 00:08:15:00:08:15
    - ether src 00:08:15:00:08:15
    - ether dst 00:08:15:00:08:15
    - not ether host 00:08:15:00:08:15

5. 특정 애플리케이션에 대한 트래픽 수집(예시)
    - port 20 or port 21
    - not port 53
    - udp port 67
    - tcp port 21
    - portrange 1-80
    - tcp portrange 1-80
    - host 10.3.1.1 and port 80
    - host 10.3.1.1 and not port 80
    - udp src port 68 and udp dst port 67

- 캡처필터란
    - 데이터 수집에 대한 제한을 두는 것
    - 캡처 필터 문법
    - 수집한 패킷의 자동 파일화 - 캡처 -> 출력에서 설정
    - 정지 방법 - 캡처 -> 옵션에서 설정 

### 디스플레이 필터
- 적절한 디스플레이 필터 문법 사용

- 간단한 디스플레이 필터 문법
    - arp
    - ip
    - ipv6
    - tcp
    1. 단순 IP 주소 호스트에게/부터의 트래픽 필터링
        - ip.addr == 10.3.1.1
        - !ip.addr ==10.3.1.1
        - ipv6.addr == 2406:da00:ff00::6b16:f02d
        - ip.src==10.3.1.1
        - ip.dst == 10.3.1.1
        - ip.host == www.wireshark.org

    2. 주소 범위에게/부터의 트래픽 필터링
        - ip.addr > 10.3.0.1 && ip.addr < 10.3.0.5 
        - (ip.addr >= 10.3.0.1 && ip.addr <= 10.3.0.6) && !ip.addr==10.3.0.3
        - ipv6.addr == fe80:: && ipv6.addr < fec0::

    3. IP 서브넷에서/으로부터 트래픽 필터링
        - CIDR 방식.
        - ip.addr == 10.3.0.0/16
        - ip.addr == 10.3.0.0/16 && !ip.addr ==10.3.0.3
        - !ip.addr == 10.3.0.0/16 && !ip.addr ==10.2.0.0/16




- 디스플레이 필터는 다양한 연산자가 활용된다.

### TShark 실습
- Wireshark 설치 시 같이 설치할 수 있음
- TCPdump와 문법이 비슷함
- 주로 TCPdump를 사용한다.

- 명령어
    - tshark -h : 버전 및 정보 확인
    - tshark -D : 수집 가능한 네트워크 목록 확인
    - tshark -i N : N번째 네트워크 트래픽 수집
    - tshark -i N -w NAME.pcapng : N번째 네트워크 NAME파일로 저장
    - tshark -r NAME.pcapng : NAME파일 읽기
    - tshark -i N -a files:N1 -b duration:S -w NAME.pcapng : N번째 파일 N1개 생성, S초마다 NAME파일로 저장
    - tshark –i N –f “tcp port 443” –w NAME.pcapng : 옵션 지정. TCP port 443인 패킷만 수집

## TCP/IP
### 단편화
- 네트워크 기기가 전송할 수 있는 최대 전송 단위
- 단편화
    - MTU가 큰 네트워크에서 MTU가 작은 네트워크로 데이터그램이 전송될 경우 데이터그램은 나누어서 보내져야 함
    - 데이터그램의 재조립은 최종목적지 호스트에 의해서만 수행
    - 재조립으로 인해 발생하는 비효율성 때문에 전송 중 재조립 안됨
    - 단편화와 관련된 필드 : Identification, Flag, Fragmentation offset

- MSS: (Maximum Segment Size)
    - TCP상에서 전송할 수 있는 사용자 데이터의 최대 크기
    - MSS = MTU - IP Header 크기 (최소 20 Byte) - TCP Header 크기 (최소20 Byte). **세그먼트의 데이터 부분의 크기**
    - 4계층 단위지만 헤더는 포함하지 않는다!
    - **MSS 사이즈 설정은 TCP 3WHS 과정에서 결정한다.**

- MTU는 3계층 최대 전송 단위로, IP헤서 + IP데이터의 크기가 1500을 넘을 수 없는 것이다.

- 이더넷 헤더와 트레일러가 포함된 L2에서는 1518바이트가 전송될 수 있다.


### UDP/TCP 단편화 차이점
- UDP
    - Message-Oriented Transport Protocol
    - 상위계층에서 payload 된 데이터를 **단편화하지 않음**
        - IP계층(L3)에서 단편화 작업 진행
        - 상위 계층의 메시지를 그대로 전송하기 때문에 Message-Oriented.
        - IP계층에서 단편화 시 IP헤더와 UDP헤더의 크기를 보고 데이터의 크기를 결정한다.
    - 데이터 전송이 완료되지 전까지는 다른 데이터 흐름에 영향을 받지 않음


- TCP
    - Stream-Oriented Transport Protocol 
    - 상위계층에서 payload 된 데이터를 **단편화**
        - MSS 크기에 맞춰 단편화 진행
        - **단편화 조건: MSS < payload data**
        - 페이로드되는 데이터의 크기가 MSS보다 커야 4계층에서 단편화를 진행한다.
            - 데이터의 크기: TCP 헤더가 붙기 전 5계층에서 내려오는 데이터 사이즈를 의미
    - 4계층에서 단편화를 진행해도 3계층에서 또 다시 단편화를 진행해야 할 수도 있다.
        - **되도록 4계층에서 단편화를 진행**하려고 함. 단, 4계층에서 단편화 진행 시 TCP데이터이어야 하고, MSS 사이즈를 고려해야 한다. 
        - **4계층에서 단편화 진행 시 재조립이 간단함(SN을 사용하기 때문에, SN를 순서대로 나열하면 재조립이된다.)**
        - 3계층에서는 ID, Offset, Flag를 다 고려해야하기 때문에 오래 걸린다. (3계층에서 단편화 시 재조립 시 오래걸리는 문제가 존재.)

### Urgent pointer
- TCP 헤더의 urg플래그와 urgent pointer의 값이 같이 저장된다.
- 기본적으로 SN이 작은 것부터 전송이 된다.
    - 긴급하게 순서를 바꿔야 하는 경우 사용하는 것이 Urgent pointer이다.
    - **Urgent pointer: 전송 우선순위 변경 필드**
    - urg 플래그가 1인 패킷들이 여러 개 존재할 경우, Urgent pointer 값이 **작은** 패킷부터 우선적으로 처리한다.

### push flag
- 전송하려는 데이터가 항상 MSS를 넘을만큼 큰 건 아니다. 그러나 전송하려는 데이터가 원래 작은 건지, 값이 손상되어 작아진건지 구분할 수 없기 때문에 원래 사이즈가 작다는 것을 표시해주는 플래그.

- 작은 용량의 데이터를 우선순위 없이 바로 내보낸다.

- MTU나 MSS와 상관 없이 전송하는 패킷들에 사용(독립적으로 작은 사이즈)

### 정리
- TCP 3WHS
    - 상태 확인: 시스템 활성화 여부, 서비스 활성화 여부
    - 흐름 제어: Window size + option(window scale), SACK Permit 공유
    - MTU, MSS 설정

- TCP
    - 상위 계층의 PDU(message)를 조각난 덩어리(stream)로 처리하기 때문에 stream-oriented transport protocol
- UDP
    - 상위 계층의 PDU(message)를 그대로 처리하기 때문에 message-oriented transport protocol

- 1518Byte: Ethernet Header + IP Header + TCP Header + TCP data
- 1500Byte: IP Header + TCP Header + TCP data
- IP header, TCP header가 각각 30byte인 경우 3계층에서 단편화를 일으키지 않는 최적의 MSS는?
    - 1500 - 60 = 1440. 최적의 MSS = 1440Byte

- MTU 변경 --> 레지스트리 편집기 또는 netsh 명령어 사용(점보 프레임 지원 시 가능)
    - C:\> netsh interface ipv4 set subinterface “번호" mtu=9000 store=persistent
    - C:\>netsh interface ipv4 show global 

### TTL (Time To Live)
- 최대 홉수 확인
    - C:\>netsh interface ipv4 show global 
- TTL 값 변경
    - 128에서 64로 변경
    - C:\> netsh interface ipv4 set global defaultcurhoplimit=64

- 설정 확인 명령어
- C:\>netsh interface show interface
- C:\>netsh interface ipv4 show global
- C:\>netsh interface ipv4 show interfaces

### Header checksum
- 헤더의 오류를 검증하기 위해 사용
- 오류 검출: 수신된 데이터의 무결성을 검증하는 과정
- IP 계층만 헤더 체크섬이 존재. (나머지는 체크섬)
- 송신자: 오류 검출 코드를 첨부
- 수신자: 첨부된 오류 검출 코드를 기반으로 무결성 점검

- 오류 검출 코드를 생성하는 방법 --> 프로토콜마다 다름
- IP에서의 생성 방법
    - 체크섬을 제외한 모든 필드 값을 16비트로 묶어 더한 값을 사용한다.
    - 더한 후 2진화하여 1의 보수를 취한 값을 체크섬으로 사용한다.

- TCP, UDP --> checksum 값에는 "헤더 + 데이터"
- IP       --> checksum 값은 헤더에 대한 무결성만 점검 가능
    - IPv6는 체크섬 사용하지 않음

### ICMP
- 3계층에서 통신이 잘 되는지 확인할 수 있는 메시지 프로토콜
- ICMP 사용 이유
    - IP Protocol은 신뢰성이 없고 비연결형 Protocol
    - IP Protocol은 에러 발생 원인이나 진단 기능 및 상황 정보를 지원하지 않음
    - --> ICMP로 커버

- UDP의 비신뢰성과 비연결성 --> TCP가 보완
- 3계층 IP프로토콜의 비신뢰성과 비연결성 --> ICMP가 보완
    - IP 데이터 부분에 ICMP 메시지 포함.
    - ICMP 메시지는 항상 앞에 IP 헤더를 달고 있다.(독립적X. IP에 기생)


- 기본적인 ping은 32바이트
    - ping -l 100 옵션을 통해 송신 사이즈를 100바이트로 확장시켜 보낼 수 있다.

### ICMP Header
- 총 8Byte: 타입, 코드, 체크섬, 기타 헤더, 데이터 영역(ICMP Body)
- **타입, 코드**는 8bit, 체크섬은 16bit
- 데이터 영역이 MTU를 넘어가면 단편화 발생
- ping -f ~: 단편화하지 않도록 설정하는 플래그를 1로 설정
    - ping -l 2000 -f ~: 단편화를 해야하는 사이즈인데, 단편화를 하지 못하도록 설정. --> 에러 발생. 단편화가 필요한데, DF가 설정되어있어 할 수 없음.
- ping -i 3 ~: TTL 3으로 설정
- TTL 값 만료로 목적지까지 못갈 경우 **패킷을 폐기한 게이트웨이가** 메시지를 보냄.

    ### ICMP 메시지 종류
    1. 오류 보고 메시지 (Error Reporting Message)
        - IP패킷 처리 도중 발생한 문제 보고

    2. 질의 메시지 (Query Message)
        - 다른 호스트로부터 특정 정보 획득
        - 네트워크 문제 진단

    - ICMP는 타입과 코드가 중요하다.
    - 질의 메시지보다 오류 메시지가 중요하다.

- ping --> ICMP --> IP의 단점을 보강한 프로토콜 --> IP Header 필드의 특정 부분을 변경해가서 테스트를 진행


## Ethernet 
- Ethernet LAN
    - Ethernet LAN 안을 돌아다니는 프레임이 두 가지가 있다.
        1. Ethernet Frame (DIX2.0)
            - 송신지 MAC 주소 뒤에 Type 필드
            - 일반적으로 호스트가 생성
            - preambl 8Byte
            
        2. IEEE 802.3 & IEEE 802.2
            - 송신지 MAC 주소 뒤에 Length 필드
            - 보통 스위치가 생성
            - preamble 7 + SOF 1 --> 8Byte

- 2계층에서의 오류는 크게 신경을 쓰지 않는다.
    - 따라서 와이어샤크 자체에서 **FCS를 숨기는 경우가 일반적**이다.
            

## ARP (Address Resolution Protocol)
- IP 주소를 MAC 주소로 변환해주는 프로토콜
- ARP request packet
- ARP Cache Table에 존재하지 않을 시 request packet 보냄

- 3계층 프로토콜

- 타입: 0x0806 --> ARP임을 나타냄

- ARP 데이터 구성
    - |       하드웨어 타입     | 프로토콜 타입         |
    - |하드웨어 길이|프로토콜 길이| 오퍼레이션            |
    - |             Sender hardware address         |(송신지 L2)
    - |             Sender protocol address         |(송신지 L3)
    - |             Target hardware address         |(수신지 L2)
    - |             Target protocol address         |(수신지 L3)


- 여러가지 망에서 호환이 가능하여 IP, MAC이라고 정하지 않는다.
- 

